In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from neo4j import GraphDatabase
from scipy import linalg
from scipy.spatial.distance import cosine
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler

In [14]:
list_subfolders_with_paths = [f.name for f in os.scandir(os.getcwd()) if f.is_dir()]
subfolders_lst = [x for x in list_subfolders_with_paths if "." not in x]
subfolders_lst

['MineralConstituentOf', 'MineralSourceFor']

In [15]:
total = []
for subdir in subfolders_lst:
    myfilename = subdir + "/ent_tuples.json"
    with open(myfilename) as f:
        df = json.load(f)
    total.append(df)

flat_total = [item for sublist in total for item in sublist]

Create unique node list for graph.

In [16]:
word_pairs = set([item for sublist in [item[0] for item in flat_total] for item in sublist])

Generate a list of all the nodes.

In [17]:
nodes = []
for item in word_pairs:
    nodes.append({"Word": item})
print("Total number of nodes:", len(nodes))

Total number of nodes: 355


Generate a list of all the relationships between nodes with probability as weight.

Connect to neo4j database.

In [19]:
url = "bolt://localhost:7687"
username = "neo4j"
password = "scribbles"

# Connect to the neo4j database server
graphDB_Driver  = GraphDatabase.driver(url, auth=(username, password))
graphDB_Driver.open

<bound method BoltDriver.open of <class 'neo4j._sync.driver.BoltDriver'>>

Run the query to create all nodes in neo4j.

In [20]:
graphDB_Driver.session().run(
    query = 'UNWIND $props AS map CREATE (n:Word) SET n = map',
    parameters = {'props': nodes}
)

Run the query to create all relationships in neo4j.

In [21]:
for i in range(len(subfolders_lst)):
    print(subfolders_lst[i])
    relationships = []
    for item in total[i]:
        relationships.append({"from":item[0][0],"to":item[0][1], "properties":{"probability":item[1]}})

    graphDB_Driver.session().run(
        query = 'UNWIND $batch as row \
        MATCH (from:Word {Word: row.from}) \
        MATCH (to:Word {Word: row.to}) \
        CREATE (from)-[rel:' + subfolders_lst[i] + ']->(to) \
        SET rel += row.properties', 
        parameters = {'batch': relationships}
    )

MineralConstituentOf
MineralSourceFor


Delete all nodes and relationships.

In [10]:
graphDB_Driver.session().run(
    query = 'match (n) detach delete n', 
    parameters = {'batch': relationships}
)

In [21]:
range(len(subfolders_lst))

range(0, 2)

In [22]:
len(total)

2

In [ ]:
match (s)-[r]->(e)
with s,e,type(r) as typ, tail(collect(r)) as coll 
foreach(x in coll | delete x)